Guidance on using this dataset: https://www.kaggle.com/code/acelevin/identifying-playing-cards

Download dataset from: https://www.kaggle.com/datasets/gunhcolab/object-detection-dataset-standard-52card-deck/data

In [1]:
import tensorflow as tf
import pickle
from PIL import Image
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [23]:
#hyper parameters:
BATCH_SIZE = 32
NUM_EPOCHS = 20

### My attempt at a ResNet Network Model

In [4]:
class ResidualBlock(tf.keras.layers.Layer):
    ''' Res Block for an attempt at a ResNet architecture clone'''
    def __init__(self, filters, downsample=False):
        super(ResidualBlock, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        self.filters = filters
        self.downsample = downsample

        # downsample will halve the image size
        strides = 2 if downsample else 1

        self.conv1 = tf.keras.layers.Conv2D(filters, 3, strides=strides, padding='same', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(1e-4))
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()

        self.conv2 = tf.keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(1e-4))
        self.bn2 = tf.keras.layers.BatchNormalization()

        # Optional downsampling for the shortcut path
        if downsample:
            self.downsample_conv = tf.keras.layers.Conv2D(filters, 1, strides=2, padding='same', use_bias=False)
            self.downsample_bn = tf.keras.layers.BatchNormalization()
        else:
            self.downsample_conv = None
            
    def call(self, inputs, training=False):
        shortcut = inputs

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        if self.downsample_conv:
            shortcut = self.downsample_conv(shortcut)
            shortcut = self.downsample_bn(shortcut, training=training)

        x = tf.keras.layers.add([x, shortcut])
        return self.relu(x)
        

class MockResNet(tf.keras.Model):
    def __init__(self):
        super(MockResNet, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [
            tf.keras.Input(shape=(300, 300, 3)),
            #data_augmentation,
            
            tf.keras.layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
            
            tf.keras.layers.Dropout(0.4),
            ResidualBlock(64),
            tf.keras.layers.Dropout(0.4),
            ResidualBlock(64),

            ResidualBlock(128, downsample=True),
            tf.keras.layers.Dropout(0.4),
            ResidualBlock(128),
            tf.keras.layers.Dropout(0.4),

            ResidualBlock(256, downsample=True),
            tf.keras.layers.Dropout(0.4),
            # ResidualBlock(256),
            # tf.keras.layers.Dropout(0.4),

            ResidualBlock(512, downsample=True),
            tf.keras.layers.Dropout(0.4),
            # ResidualBlock(512),
            # tf.keras.layers.Dropout(0.3),

            tf.keras.layers.GlobalAveragePooling2D(),
            
            #tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(52, activation='softmax')
        ]
        self.sequential = tf.keras.Sequential(self.architecture)
    
    def call(self, inputs, training=False):
        return self.sequential(inputs, training=training)
        
    @staticmethod
    def loss_fn(labels, predictions): 
           """ Loss function for the model. """
           return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

### Traditional CNN Model

In [5]:
class CardPredictor(tf.keras.Model):
    def __init__(self):
        super(CardPredictor, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [        
                tf.keras.layers.InputLayer((300, 300, 3)),
                data_augmentation,
                
                tf.keras.layers.Conv2D(32, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.Conv2D(32, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Dropout(0.2),
                
                tf.keras.layers.Conv2D(64, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.Conv2D(64, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Dropout(0.3),
                
                tf.keras.layers.Conv2D(128, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.Conv2D(128, 3, padding='same', use_bias=False),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Dropout(0.4),
                
                tf.keras.layers.GlobalAveragePooling2D(),
                
                tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(52, activation='softmax')        
                ]
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor")
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

    @staticmethod
    def loss_fn(labels, predictions): 
           """ Loss function for the model. """
           return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

### Dataset Loading

In [6]:
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)
    
new_data = {}
for key, inner_dict in data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['class_label']
    new_data[img_path] = value

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((new_data.keys(), new_data.values()))

def load_train_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = dataset.map(load_train_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [8]:
# this cell it optional
# this checks that the datasets  is balanced
import collections
v_counts = collections.Counter()
t_counts = collections.Counter()


for _, label in val_dataset:
    label = label.numpy().tolist() 
    v_counts.update(label)

for _, label in train_dataset:
    label = label.numpy().tolist() 
    t_counts.update(label)
    
print("Label counts in validation dataset:")
for i in range(52):
    print(f"Label {i}: {v_counts[i]}, {t_counts[i]}")

2025-04-27 13:44:43.015215: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Label counts in validation dataset:
Label 0: 29, 106
Label 1: 30, 107
Label 2: 27, 108
Label 3: 23, 103
Label 4: 35, 105
Label 5: 30, 113
Label 6: 28, 111
Label 7: 27, 110
Label 8: 29, 109
Label 9: 28, 110
Label 10: 22, 113
Label 11: 24, 114
Label 12: 33, 94
Label 13: 25, 115
Label 14: 27, 111
Label 15: 21, 113
Label 16: 28, 103
Label 17: 26, 114
Label 18: 27, 107
Label 19: 22, 102
Label 20: 26, 95
Label 21: 27, 113
Label 22: 31, 110
Label 23: 28, 101
Label 24: 25, 108
Label 25: 34, 101
Label 26: 28, 101
Label 27: 25, 101
Label 28: 31, 105
Label 29: 27, 110
Label 30: 22, 108
Label 31: 31, 113
Label 32: 30, 108
Label 33: 23, 110
Label 34: 26, 109
Label 35: 22, 109
Label 36: 27, 108
Label 37: 27, 105
Label 38: 23, 106
Label 39: 28, 109
Label 40: 17, 108
Label 41: 30, 112
Label 42: 26, 110
Label 43: 26, 108
Label 44: 22, 109
Label 45: 29, 102
Label 46: 30, 99
Label 47: 31, 109
Label 48: 17, 110
Label 49: 26, 107
Label 50: 30, 106
Label 51: 34, 106


2025-04-27 13:44:50.076101: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Model Loading and Training

In [25]:
model = CardPredictor()
#model = MockResNet()
#model = tf.keras.models.load_model("model_weights_89.keras")
model.compile(optimizer=model.optimizer, loss=model.loss_fn, metrics=['accuracy'])

In [33]:
model.fit(train_dataset, 
          validation_data=val_dataset, 
          epochs=NUM_EPOCHS, 
          verbose=1)

Epoch 1/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 44s 209ms/step - accuracy: 0.8509 - loss: 10751.3350 - val_accuracy: 0.8620 - val_loss: 9784.5518
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 44s 214ms/step - accuracy: 0.8442 - loss: 13309.9189 - val_accuracy: 0.8815 - val_loss: 8711.1084
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 45s 216ms/step - accuracy: 0.8650 - loss: 11565.9199 - val_accuracy: 0.8721 - val_loss: 11484.7246
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 46s 214ms/step - accuracy: 0.8758 - loss: 10543.7910 - val_accuracy: 0.8699 - val_loss: 11041.3184
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 44s 212ms/step - accuracy: 0.8615 - loss: 12944.3428 - val_accuracy: 0.8577 - val_loss: 16974.2871
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 44s 211ms/step - accuracy: 0.8621 - loss: 15146.7959 - val_accuracy: 0.8671 - val_loss: 14851.2021
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 45s 216ms/step - accuracy: 0.8485 - loss: 17245.9980 - val_accuracy: 0.8707 - val_loss: 14440.5352
Epoch 8/20
175/175 ━━━━━━━━━━

In [32]:
model.save_weights('model_weights.weights.h5')